### Exploring GNNs for molecular property prediction

In [8]:
from gnn import *
import tensorflow as tf
import nfp
print(tf.__version__)
print(nfp.__version__)

2.5.0
0.3.0


In [13]:
# Check for gpus
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if len(gpus) > 0:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    device = "/gpu:0"
    print('Using GPU 0')
else:
    device = "/cpu:0"
    print('No GPU found, falling back to CPU')
    
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['TF_MKL_REUSE_PRIMITIVE_MEMORY'] = '0'

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Using GPU 0


In [65]:
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
import nfp
import json 
import sys
from datetime import date

In [16]:
from sklearn.model_selection import KFold

In [18]:
random_seed = 42 # Reproducible splits

#### Let's run an initial training iteration to see how the model works

In [39]:
data = pd.read_csv('./molecules_to_predict.csv')
weighting_factor = 0.6
data

,Canonical_SMILES,Device_tier,Train/Valid/Test,CN,predicted,glob_vector
0,CCCCCCCCC=CCCCCCCCC(=O)OC,1,Train,56.000,57.968243,2.691358 14.592575 -9.436014 -5.058424 3.00276...
1,CCCCC/C=C\C/C=C\CCCCCCCC(=O)O,1,Train,31.400,32.391766,1.516829 8.199537 -5.012876 -3.377008 1.524693...
2,CCCCC1CCCC2CCCCC12,1,Train,30.815,29.558361,1.745941 7.646197 -4.469603 -2.352664 1.533207...
3,CC(=CCC=C(C)C=C)C,1,Train,28.000,28.981424,1.265745 7.677176 -4.606571 -2.230392 1.572281...
4,CCCCCCCC(=O)OCC,1,Train,42.060,41.965744,1.682150 10.854990 -6.497653 -3.748828 2.38611...
...,...,...,...,...,...,...
625,CCCCCCCCCCCCCCCCCC(=O)O,3,Test,62.000,70.268470,4.218513 18.318495 -10.543981 -6.405912 4.1426...
626,C1(=C(C=C(C=C1C)C)CCCCCCCCCCCCCCCC)C,3,Test,42.000,38.849712,2.940197 9.780895 -6.842910 -3.857143 3.013691...
627,CCCCCCOC(=O)C1=CC=CC=C1C(=O)OCCCCCC,3,Test,48.000,46.116280,1.885477 14.634890 -7.869594 -3.706099 2.03854...
628,CCCC1=CC2=C(CCCC2)C=C1,3,Test,8.000,7.196462,0.202410 1.829508 -1.479237 -0.212674 0.350839...


Note the *Device_tier* column - this is a measure of experimental reliability, and is unique to this model. Later, when defining our dataset, we'll tell the model to weight the lower tier (2 and 3) data points less when using these data points to calculate model weights and biases, per the weighting factor parameter above.

We'll split the dataset into three parts: a training set, a validation set, and a test set.  
The training set represents data that the model uses to learn the weights and biases that allow the neural net to make predictions.  
The validation set is used to make an evaluation of the goodness-of-fit of the model during the training process.  
The test set is used to evaluate the final model on a sample set of data that is representative of the population the model will be used to make predictions on.  
  
It may seem strange to have both a validation set and a test set - this is due to the fact that the test set must remain totally unseen. Ie., to avoid introducing human bias, the test set should not be used to select things like model hyperparameters during training. This role is then filled by the validation test. In the literature, you'll sometimes see these two terms used interchangably - this is technically not correct.

In [19]:
train = data.sample(frac=.8, random_state=random_seed)
valid = data[~data.index.isin(train.index)].sample(frac=.5, random_state=random_seed)
test = data[~data.index.isin(train.index) & ~data.index.isin(valid.index)]

In [20]:
preprocessor = CustomPreprocessor(
    explicit_hs=False,
    atom_features=atom_features,
    bond_features=bond_features)

The preprocessor object reads a data object, parses it into features used by the neural network to describe the unique chemical neighborhoods of each molecule, and categorizes atoms and bonds based on these features. Let's take a look at what its doing, using a test molecule

In [30]:
mol = rdkit.Chem.MolFromSmiles('CO')
print(atom_features(next(mol.GetAtoms()))) # The rdkit GetAtoms method returns an iterator of rdkit Atom objects; get the next item using next keyword.

(3, 'C', False, 0)


In [31]:
print(bond_features(next(mol.GetBonds())))

(rdkit.Chem.rdchem.BondType.SINGLE, 0, 'C', 'O')


In [32]:
print(f"Atom classes before: {preprocessor.atom_classes} (includes 'none' and 'missing' classes)")
print(f"Bond classes before: {preprocessor.bond_classes} (includes 'none' and 'missing' classes)")

Atom classes before: 2 (includes 'none' and 'missing' classes)
Bond classes before: 2 (includes 'none' and 'missing' classes)


In [34]:
for smiles in train.Canonical_SMILES:
    preprocessor.construct_feature_matrices(smiles, train=True)
print(f'Number of unique atom classes found after preprocessing: {preprocessor.atom_classes}')
print(f'Number of unique bond classes found after preprocessing: {preprocessor.bond_classes}')

Number of unique atom classes found after preprocessing: 26
Number of unique bond classes found after preprocessing: 23


The tensorflow model needs to know how the model output looks:

In [36]:
output_signature = (preprocessor.output_signature,
                        tf.TensorSpec(shape=(), dtype=tf.float32),
                        tf.TensorSpec(shape=(), dtype=tf.float32))

In [41]:
train_data = tf.data.Dataset.from_generator(
    lambda: create_tf_dataset(train, preprocessor, weighting_factor, True), output_signature=output_signature)\
    .cache().shuffle(buffer_size=1000)\
    .padded_batch(batch_size=16)\
    .prefetch(tf.data.experimental.AUTOTUNE)

The batch size is a parameter that allows for more efficient training - by batching samples, tensorflow is able to take advantage of the larger memory and parallel processing abilities of modern GPUs. In essence, tensorflow stacks inputs on top of each other (number given by the batch size argument) and performs each training iteration for the entire batch, as opposed to just one input at a time. So, if each input looks like a [32, 32] matrix, a batch size of 16 will create a batch object of size [32, 32, 16]. This can greatly speed up training, but this number should be tweaked to fit the GPU you're working with. If this is too high, it can cause Out of Memory errors.

In [43]:
valid_data = tf.data.Dataset.from_generator(
    lambda: create_tf_dataset(valid, preprocessor, weighting_factor, False), output_signature=output_signature)\
    .cache()\
    .padded_batch(batch_size=16)\
    .prefetch(tf.data.experimental.AUTOTUNE)
test_data = tf.data.Dataset.from_generator(
    lambda: create_tf_dataset(test, preprocessor, weighting_factor, False), output_signature=output_signature)\
    .cache()\
    .padded_batch(batch_size=16)\
    .prefetch(tf.data.experimental.AUTOTUNE)


In [44]:
atom_Input = layers.Input(shape=[None], dtype=tf.int32, name='atom')
bond_Input = layers.Input(shape=[None], dtype=tf.int32, name='bond')
connectivity_Input = layers.Input(shape=[None, 2], dtype=tf.int32, name='connectivity')
global_Input = layers.Input(shape=[2], dtype=tf.float32, name='mol_features')

In [45]:
features_dim = 64
num_messages = 5

In [48]:
atom_state = layers.Embedding(preprocessor.atom_classes, features_dim,
                              name='atom_embedding', mask_zero=True,
                              embeddings_regularizer='l2')(atom_Input)

In [46]:
bond_state = layers.Embedding(preprocessor.bond_classes, features_dim,
                              name='bond_embedding', mask_zero=True,
                              embeddings_regularizer='l2')(bond_Input)

In [47]:
global_state = layers.Dense(features_dim, activation='relu')(global_Input) 

In [49]:
for i in range(num_messages):
    atom_state, bond_state, global_state = message_block(atom_state, bond_state,
                                                            global_state, connectivity_Input, features_dim, i)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [50]:
prediction = layers.Dense(1)(global_state)

In [51]:
input_tensors = [atom_Input, bond_Input, connectivity_Input, global_Input]

In [52]:
model = tf.keras.Model(input_tensors, [prediction])

In [54]:
learning_rate = 1.0e-4
model.compile(loss='mae', optimizer=tf.keras.optimizers.Adam(learning_rate))

In [67]:
cal = date.today()
model_path = f'./model_files/{cal.strftime("%Y%m%d")}/{cal.strftime("%H%M")}.hf'
checkpoint = ModelCheckpoint(model_path, monitor="val_loss",\
                                 verbose=2, save_best_only = True, mode='auto', period=1 )

In [68]:
hist = model.fit(train_data,
                     validation_data=valid_data,
                     epochs=20,
                     verbose=2, callbacks = [checkpoint])

Epoch 1/20


2023-06-05 11:24:33.670895: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-06-05 11:24:33.716386: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2400000000 Hz
2023-06-05 11:24:36.016337: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-06-05 11:24:36.434809: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


32/32 - 8s - loss: 26.9923 - val_loss: 26.2861

Epoch 00001: val_loss improved from inf to 26.28610, saving model to ./model_files/20230605/0000.hf


2023-06-05 11:24:40.448409: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/home/sabari/.conda/envs/cn-pred/lib/python3.7/site-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: ./model_files/20230605/0000.hf/assets


...

INFO:tensorflow:Assets written to: ./model_files/20230605/0000.hf/assets


Epoch 19/20
32/32 - 0s - loss: 7.9536 - val_loss: 12.5060

Epoch 00019: val_loss did not improve from 10.50139
Epoch 20/20
32/32 - 0s - loss: 7.5705 - val_loss: 9.8098

Epoch 00020: val_loss improved from 10.50139 to 9.80983, saving model to ./model_files/20230605/0000.hf


/home/sabari/.conda/envs/cn-pred/lib/python3.7/site-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: ./model_files/20230605/0000.hf/assets


INFO:tensorflow:Assets written to: ./model_files/20230605/0000.hf/assets


In [75]:
# Load in the trained model weights. Replace this with a path to the downloaded fully trained weights if
# you want to make actual predictions
model.load_weights(model_path)

2023-06-05 12:42:43.246091: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open ./model_files/20230605/0000.hf: Failed precondition: model_files/20230605/0000.hf; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


In [71]:
train_results = model.predict(train_data).squeeze()
valid_results = model.predict(valid_data).squeeze()

mae_train = np.abs(train_results - train['CN']).mean()
mae_valid = np.abs(valid_results - valid['CN']).mean()

In [73]:
preprocessor.to_json(f"model_files/{cal.strftime('%Y%m%d')}/preprocessor.json")

In [74]:
mae_train

27.107588859568853